In [1]:
!pip install qdrant-client
!pip install sentence_transformers

  Using cached qdrant_client-1.11.0-py3-none-any.whl.metadata (10 kB)
  Using cached grpcio_tools-1.65.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached protobuf-5.27.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached grpcio-1.65.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.3 kB)
Using cached qdrant_client-1.11.0-py3-none-any.whl (258 kB)
Using cached grpcio_tools-1.65.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.3 MB)
Using cached grpcio-1.65.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.7 MB)
Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
Using cached protobuf-5.27.3-cp38-abi3-manylinux2014_x86_64.whl (309 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4
  Attem

In [2]:
!pip install python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [106]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
import json
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()



True

In [97]:
with open('diabetes_data_with_vectors', 'r') as f_in:
    diabetes_data_with_vectors = json.load(f_in)

In [119]:
api_key = os.getenv('QDRANT_API_KEY')


client = QdrantClient(
    url="https://8999b86c-f8b2-4d60-bdfa-8c68d39daae7.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    api_key=api_key,timeout=200
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='diabetes')]


In [120]:
client.delete_collection(
    collection_name="diabetes")

True

In [121]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="diabetes",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [122]:
from qdrant_client.http.models import PointStruct



# Upsert data in chunks
#for chunk in chunk_data(diabetes_data_with_vectors, chunk_size):
client.upsert(
    collection_name="diabetes",
    points=[
        PointStruct(
            id=idx,
            vector=data['qa_text_embeddings'],
            payload={"question": data['question'], "answer": data['answer'], "rand_number": idx % 10}
        )
        for idx, data in enumerate(diabetes_data_with_vectors)
    ]
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [123]:
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')


In [128]:
query_vector = embedding_model.encode('What are the recommended food for a diabetes patients').tolist()

In [129]:
hits = client.search(
    collection_name="diabetes",
    query_vector=query_vector,
    limit=5  # Return 5 closest points
)

In [130]:
collection_name = "diabetes"
count = client.count(collection_name=collection_name)

print(f"Total number of points in the collection '{collection_name}': {count}")

Total number of points in the collection 'diabetes': count=1071


In [131]:
for hit in hits:
    print(hit.payload)

{'question': 'What types of foods are emphasized in a healthy eating plan for diabetes?', 'answer': 'A healthy eating plan for diabetes emphasizes a variety of foods including breads, cereals, rice, whole grains, fruits, vegetables, meat and meat substitutes, dairy products, and healthy fats.', 'rand_number': 7}
{'question': 'Question: Can you list some specific examples of foods rich in healthy fats that are suitable for individuals with diabetes?', 'answer': 'Answer:  Yes, some great examples of healthy fat-rich foods suitable for individuals with diabetes include olive oil, avocados, almonds, walnuts, salmon, tuna, and flaxseeds. These foods provide essential nutrients and support overall well-being.', 'rand_number': 6}
{'question': 'What are the key components of a healthy eating plan for diabetes?', 'answer': 'A healthy eating plan for diabetes emphasizes breads, cereals, rice, whole grains, fruits, vegetables, meat and meat substitutes, dairy products, and healthy fats, while foc